In [20]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [7]:
#specify the path to the dataset
traindir = r"C:\Users\totol\Documents\dataset\train"
valdir = r"C:\Users\totol\Documents\dataset\val"

In [8]:
def extract_url_labels(dir):
    #create an empty pandas dataframe
    df = pd.DataFrame({'url': [], 'label': []})
    actions_dir = listdir(dir)
    for tmpdir in actions_dir:
        #print(tmpdir)
        Lfiles = listdir(dir+"\\"+tmpdir)
        #print(Lfiles)
        for video in Lfiles:
            df.loc[len(df)] = [dir + "\\"+ video, tmpdir]
    return(df)


In [9]:
train_data =extract_url_labels(traindir)
train_data


,url,label
0,C:\Users\totol\Documents\dataset\train\-dJCAuk...,applauding
1,C:\Users\totol\Documents\dataset\train\-EP-5Lq...,applauding
2,C:\Users\totol\Documents\dataset\train\-LXRdUM...,applauding
3,C:\Users\totol\Documents\dataset\train\-PgjjBi...,applauding
4,C:\Users\totol\Documents\dataset\train\-r8c7F4...,applauding
...,...,...
8696,C:\Users\totol\Documents\dataset\train\_gLNQ0e...,washing dishes
8697,C:\Users\totol\Documents\dataset\train\_PBVM5E...,washing dishes
8698,C:\Users\totol\Documents\dataset\train\_Q9DzFp...,washing dishes
8699,C:\Users\totol\Documents\dataset\train\_zgt8QO...,washing dishes


In [10]:
val_data = extract_url_labels(valdir)
val_data

,url,label
0,C:\Users\totol\Documents\dataset\val\KGAGYJI51...,applauding
1,C:\Users\totol\Documents\dataset\val\KMHDg1rBQ...,applauding
2,C:\Users\totol\Documents\dataset\val\kMM5v77Tu...,applauding
3,C:\Users\totol\Documents\dataset\val\Kn9-3lPzS...,applauding
4,C:\Users\totol\Documents\dataset\val\koecwPF1b...,applauding
...,...,...
481,C:\Users\totol\Documents\dataset\val\xJD73jh8U...,washing dishes
482,C:\Users\totol\Documents\dataset\val\YDIk6Lmlv...,washing dishes
483,C:\Users\totol\Documents\dataset\val\zaKK2ZjRR...,washing dishes
484,C:\Users\totol\Documents\dataset\val\ZnjpSYYDh...,washing dishes


In [15]:
print("nobre de vidéos au total (train+validation) : "+ str(len(train_data)+len(val_data)))

nobre de vidéos au total (train+validation) : 9187


In [16]:
def format_frames(frame, output_size):
  """
    Pad and resize an image from a video.
    
    Args:
      frame: Image that needs to resized and padded. 
      output_size: Pixel size of the output frame image.

    Return:
      Formatted frame with padding of specified output size.
  """
  frame = tf.image.convert_image_dtype(frame, tf.float32)
  frame = tf.image.resize_with_pad(frame, *output_size)
  return frame

In [17]:
def frames_from_video_file(video_path, n_frames, output_size = (224,224), frame_step = 15):
  """
    Creates frames from each video file present for each category.

    Args:
      video_path: File path to the video.
      n_frames: Number of frames to be created per video file.
      output_size: Pixel size of the output frame image.

    Return:
      An NumPy array of frames in the shape of (n_frames, height, width, channels).
  """
  # Read each video frame by frame
  result = []
  src = cv2.VideoCapture(str(video_path))  

  video_length = src.get(cv2.CAP_PROP_FRAME_COUNT)

  need_length = 1 + (n_frames - 1) * frame_step

  if need_length > video_length:
    start = 0
  else:
    max_start = video_length - need_length
    start = random.randint(0, max_start + 1)

  src.set(cv2.CAP_PROP_POS_FRAMES, start)
  # ret is a boolean indicating whether read was successful, frame is the image itself
  ret, frame = src.read()
  result.append(format_frames(frame, output_size))

  for _ in range(n_frames - 1):
    for _ in range(frame_step):
      ret, frame = src.read()
    if ret:
      frame = format_frames(frame, output_size)
      result.append(frame)
    else:
      result.append(np.zeros_like(result[0]))
  src.release()
  result = np.array(result)[..., [2, 1, 0]]

  return result

In [18]:
video_path=train_data['url'][1]

In [19]:
sample_video = frames_from_video_file(video_path, n_frames = 10)
sample_video.shape

NameError: name 'cv2' is not defined